In [2]:
import json
import requests
import pandas as pandas
import re
import os 
import numpy as np
import time 
from bs4 import BeautifulSoup as soup
import sys
base='https://www.macys.com'

In [3]:
user_agent_list = [
   #Chrome
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 5.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    #Firefox
    'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1)',
    'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 6.2; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.0; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; WOW64; Trident/6.0)',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; Trident/6.0)',
    'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0; .NET CLR 2.0.50727; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729)'
]

In [4]:
# macy_page_1
big_list=[]
review_list=[]
for big_link in ['https://www.macys.com/shop/mens-clothing/all-mens-clothing/Pageindex/{}?id=197651'.format(j) for j in range(1,324)]:
    agent = {"User-Agent":user_agent_list[0]}
    try:
        page = requests.get(big_link, headers=agent)
    except:
        print('fail at {}'.format(big_link))
        continue
    page_soup=soup(page.content,'html.parser')
    links = [a['href'] for a in page_soup.find_all('a', href=True)]
    product_list=[]
    for i in links:
        if '/product/' in i:
            product_list.append(i)       
    final=[k for n in [product_list[w:w+1] for w in range(0,len(product_list),2) ] for k in n]
    print('link generated {}'.format(big_link))
    for url in list(map(lambda x: base+x, final )):
        small_list=[]
        r=requests.get(url,headers=agent)
        page_soup=soup(r.content, 'html.parser')
        try:
            product=url.split('/product/')[1].split('?ID')[0]
        except:
            product=np.nan
        try:
            review=page_soup.find_all('a',{'data-auto':'review-count'})[0].string.strip()
        except:
            review=np.nan
        try:
            price=page_soup.find_all('div',{'class':'price'})[0].string.strip()
        except:
            price=np.nan
        try:
            percent_off=page_soup.find_all('span',{'class':'c-red medium-font bold on-sale','data-auto':'sale-price'})[0].find('span').string
        except:
            percent_off=np.nan
        try:
            saleprice=str(page_soup.find_all('span',{'class':'c-red medium-font bold on-sale','data-auto':'sale-price'})[1]).split('\n')[1].strip()
        except:
            saleprice=np.nan
        try:
            sale_end=page_soup.find_all('span',text=re.compile('Sale'))[0].string
        except:
            sale_end=np.nan
        small_list.append(product)
        small_list.append(review)
        small_list.append(price)
        small_list.append(percent_off)
        small_list.append(saleprice)
        small_list.append(sale_end)
        big_list.append(small_list)
        try:
            data=page_soup.find_all('script',{'type':'application/ld+json'})
        except:
            data=[np.nan]
            print('Link {} is not working'.format(url))
            pass
        try:
            ojson = json.loads(data[0].text)["reviews"]
            review_list.append({product:ojson})
            print('review scraped {}'.format(url))
        except:
            review_list.append({product:ojson})
            print('review failed {}'.format(url))
            pass
        print('finished {}'.format(url))
    print('finished {}'.format(big_link))
    time.sleep(1)

link generated https://www.macys.com/shop/mens-clothing/all-mens-clothing/Pageindex/1?id=197651


NameError: name 'ojson' is not defined

In [ ]:
pd.DataFrame(big_list).to_csv('basic_info')

# Clean the review and create 4 table


In [ ]:
big_author=[]
big_review=[]
big_rating=[]
big_review_general=[]
fail=[]
for k in range(len(review_list)):
    author=[]
    review=[]
    rating=[]
    review_general=[]
    for i in review_list[k][list(review_list[k].keys())[0]]:
       
        author.append(i['author']['name'])
        review.append(i['description'])
        rating.append(i['reviewRating']['ratingValue'])
        review_general.append(i['reviewRating'])
    big_author.append(author)
    big_review.append(review)
    big_rating.append(rating)
    big_review_general.append(review_general)


In [ ]:
review_table=pd.DataFrame(big_review)
author_table=pd.DataFrame(big_author)
rating_table=pd.DataFrame(big_rating)
review_ge_table=pd.DataFrame(big_review_general)

In [ ]:
product_list_final=[]
for i in review_list:
    product_list_final.append(list(i.keys())[0])

In [ ]:
review_table.index=product_list_final
author_table.index=product_list_final
rating_table.index=product_list_final
review_ge_table.index=product_list_final

In [ ]:
# change to your own path before run it 
path=r'C:\Users\yzhhu\Desktop\wm fall2019\Untitled Folder'

review_table.to_csv(path+'/'+'review.csv')
author_table.to_csv(path+'/'+'author.csv')
rating_table.to_csv(path+'/'+'rating.csv')
review_ge_table.to_csv(path+'/'+'general.csv')

# Parallel 

In [2]:
from joblib import Parallel, delayed
import multiprocessing
manager = multiprocessing.Manager()

review_list = manager.list()
big_list=manager.list()


In [9]:
def pl (big_link):
    sys.setrecursionlimit(1000000)
    agent = {"User-Agent":user_agent_list[0]}
    try:
        page = requests.get(big_link, headers=agent)
    except:
        print('fail at {}'.format(big_link))
        #continue
    page_soup=soup(page.content,'html.parser')
    links = [a['href'] for a in page_soup.find_all('a', href=True)]
    product_list=[]
    for i in links:
        if '/product/' in i:
            product_list.append(i)       
    final=[k for n in [product_list[w:w+1] for w in range(0,len(product_list),2) ] for k in n]
    print('link generated {}'.format(big_link))
    for url in list(map(lambda x: base+x, final )):
        small_list=[]
        r=requests.get(url,headers=agent)
        page_soup=soup(r.content, 'html.parser')
        try:
            product=url.split('/product/')[1].split('?ID')[0]
        except:
            product=np.nan
        try:
            review=page_soup.find_all('a',{'data-auto':'review-count'})[0].string.strip()
        except:
            review=np.nan
        try:
            price=page_soup.find_all('div',{'class':'price'})[0].string.strip()
        except:
            price=np.nan
        try:
            percent_off=page_soup.find_all('span',{'class':'c-red medium-font bold on-sale','data-auto':'sale-price'})[0].find('span').string
        except:
            percent_off=np.nan
        try:
            saleprice=str(page_soup.find_all('span',{'class':'c-red medium-font bold on-sale','data-auto':'sale-price'})[1]).split('\n')[1].strip()
        except:
            saleprice=np.nan
        try:
            sale_end=page_soup.find_all('span',text=re.compile('Sale'))[0].string
        except:
            sale_end=np.nan
        small_list.append(product)
        small_list.append(review)
        small_list.append(price)
        small_list.append(percent_off)
        small_list.append(saleprice)
        small_list.append(sale_end)
        big_list.append(small_list)
        try:
            data=page_soup.find_all('script',{'type':'application/ld+json'})
        except:
            data=[np.nan]
            print('Link {} is not working'.format(url))
            pass
        try:
            ojson = json.loads(data[0].text)["reviews"]
            review_list.append({product:ojson})
            print('review scraped {}'.format(url))
        except:
            review_list.append({product:ojson})
            print('review failed {}'.format(url))
            pass
        print('finished {}'.format(url))
    print('finished {}'.format(big_link))
    time.sleep(1)

In [10]:
start=time.time()
num_cores=multiprocessing.cpu_count()
Parallel(n_jobs=num_cores)(delayed(pl)(n) for n in ['https://www.macys.com/shop/mens-clothing/all-mens-clothing/Pageindex/{}?id=197651'.format(j) for j in range(1,324)])
print('------time used: {} ------------'.format(time.time()-start))

exception calling callback for <Future at 0x1982c1ed550 state=finished raised TerminatedWorkerError>
Traceback (most recent call last):
  File "C:\Users\yzhhu\AppData\Local\Continuum\anaconda3\lib\site-packages\joblib\externals\loky\_base.py", line 625, in _invoke_callbacks
    callback(self)
  File "C:\Users\yzhhu\AppData\Local\Continuum\anaconda3\lib\site-packages\joblib\parallel.py", line 309, in __call__
    self.parallel.dispatch_next()
  File "C:\Users\yzhhu\AppData\Local\Continuum\anaconda3\lib\site-packages\joblib\parallel.py", line 731, in dispatch_next
    if not self.dispatch_one_batch(self._original_iterator):
  File "C:\Users\yzhhu\AppData\Local\Continuum\anaconda3\lib\site-packages\joblib\parallel.py", line 759, in dispatch_one_batch
    self._dispatch(tasks)
  File "C:\Users\yzhhu\AppData\Local\Continuum\anaconda3\lib\site-packages\joblib\parallel.py", line 716, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)
  File "C:\Users\yzhhu\AppData\Local\Cont

TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.

In [12]:
list(review_list)

[]